*Note: DO NOT run this notebook if you have the updated SQL table data. Please proceed to graph_setup for next steps instead.* 

## Environment setup

In [1]:
import pandas as pd
import numpy as np
import csv
import psycopg2
import math

In [2]:
#
# function to run a select query and return rows in a pandas dataframe
# pandas puts all numeric values from postgres to float
# if it will fit in an integer, change it to integer
#

def my_select_query_pandas(query, rollback_before_flag, rollback_after_flag):
    "function to run a select query and return rows in a pandas dataframe"
    
    if rollback_before_flag:
        connection.rollback()
    
    df = pd.read_sql_query(query, connection)
    
    if rollback_after_flag:
        connection.rollback()
    
    # fix the float columns that really should be integers
    
    for column in df:
    
        if df[column].dtype == "float64":

            fraction_flag = False

            for value in df[column].values:
                
                if not np.isnan(value):
                    if value - math.floor(value) != 0:
                        fraction_flag = True

            if not fraction_flag:
                df[column] = df[column].astype('Int64')
    
    return(df)

In [3]:
connection = psycopg2.connect(
    user = "postgres",
    password = "ucb",
    host = "postgres",
    port = "5432",
    database = "postgres"
)

In [4]:
cursor = connection.cursor()

## Exploring Columns and Structure

In [5]:
# Read MetObjects.txt to csv

df = pd.read_csv('MetObjects.txt')

/opt/conda/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (5,7,10,11,12,13,14,34,35,36,37,38,39,40,41,42,43,44,45,46) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
# Display all columns

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [7]:
# Display first 5 lines

df.head()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
0,1979.486.1,False,False,False,1,NaN,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,16429,Maker,,James Barton Longacre,"American, Delaware County, Pennsylvania 1794–1...",,"Longacre, James Barton",American,1794,1869,NaN,http://vocab.getty.edu/page/ulan/500011409,https://www.wikidata.org/wiki/Q3806459,1853,1853,1853,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1979",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/1,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
1,1980.264.5,False,False,False,2,NaN,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,107,Maker,,Christian Gobrecht,1785–1844,,"Gobrecht, Christian",American,1785,1844,NaN,http://vocab.getty.edu/page/ulan/500077295,https://www.wikidata.org/wiki/Q5109648,1901,1901,1901,Gold,Dimensions unavailable,"Gift of Heinz L. Stoppelmann, 1980",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/2,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
2,67.265.9,False,False,False,3,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/3,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
3,67.265.10,False,False,False,4,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/4,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
4,67.265.11,False,False,False,5,NaN,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1909–27,1909,1927,Gold,Diam. 11/16 in. (1.7 cm),"Gift of C. Ruxton Love Jr., 1967",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search/5,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN


In [8]:
# Display last 5 lines

df.tail()

,Object Number,Is Highlight,Is Timeline Work,Is Public Domain,Object ID,Gallery Number,Department,AccessionYear,Object Name,Title,Culture,Period,Dynasty,Reign,Portfolio,Constituent ID,Artist Role,Artist Prefix,Artist Display Name,Artist Display Bio,Artist Suffix,Artist Alpha Sort,Artist Nationality,Artist Begin Date,Artist End Date,Artist Gender,Artist ULAN URL,Artist Wikidata URL,Object Date,Object Begin Date,Object End Date,Medium,Dimensions,Credit Line,Geography Type,City,State,County,Country,Region,Subregion,Locale,Locus,Excavation,River,Classification,Rights and Reproduction,Link Resource,Object Wikidata URL,Metadata Date,Repository,Tags,Tags AAT URL,Tags Wikidata URL
484951,55.621.134,False,False,False,900605,NaN,Drawings and Prints,1955,Print,Holiday Card,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Engraving,Block: 3 15/16 × 2 1/2 in. (10 × 6.4 cm)\r\nSh...,"Gift of the Estate of John Taylor Arms, 1955",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484952,1977.646,False,False,False,900606,NaN,Drawings and Prints,1977,Print,Brooklyn Local,NaN,NaN,NaN,NaN,NaN,22488,Artist,,Douglas Gorsline,"American, Rochester, New York 1913–1985 Dijon,...",,"Gorsline, Douglas",American,1913,1985,NaN,NaN,NaN,1944,1939,1949,Etching,Plate: 8 1/4 × 6 7/8 in. (20.9 × 17.5 cm)\r\nS...,"John B. Turner Fund, 1977",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484953,33.40.1,False,False,False,900633,NaN,Drawings and Prints,1933,Print,Yesterday and Today,NaN,NaN,NaN,NaN,NaN,9738,Artist,,Edmond Mario Granville,"American, Chicago 1905–1969",,"Granville, Edmond Mario",American,1905,1969,NaN,NaN,NaN,ca. 1925–33,1920,1938,Etching,Plate: 8 3/4 × 5 7/8 in. (22.2 × 15 cm)\r\nShe...,"Gift of E. Mario Granville, 1933",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484954,170.1 C42,True,False,False,900717,NaN,The Libraries,NaN,NaN,"De la loi du contraste simultané des couleurs,...",NaN,NaN,NaN,NaN,NaN,19552,Author,,Michel Eugène Chevreul,1786-1889,,"Chevreul, Michel Eugène",,1786,1889,NaN,(not assigned),https://www.wikidata.org/wiki/Q271651,1839,1839,1839,NaN,"2 pages, 30 unnumbered leaves of plates (some ...",NaN,NaN,Paris,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN
484955,17.3.3457,False,False,False,900748,NaN,Drawings and Prints,1917,Print,Madison Square - Fifth Avenue and Broadway at ...,NaN,NaN,NaN,NaN,NaN,10040,Artist,,Frank M. Gregory,"American, born 1848",,"Gregory, Frank M.",American,1848,1948,NaN,NaN,NaN,1887,1887,1887,Etching,Image: 6 5/8 × 11 3/4 in. (16.9 × 29.9 cm)\r\n...,"Harris Brisbane Dick Fund, 1917",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Prints,NaN,http://www.metmuseum.org/art/collection/search...,NaN,NaN,"Metropolitan Museum of Art, New York, NY",NaN,NaN,NaN


In [9]:
# Display count of columns and rows

df.shape

(484956, 54)

In [10]:
# Rename columns
df = df.rename(columns = lambda x: x.lower().replace(' ','_'))

In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 484956 entries, 0 to 484955
Data columns (total 54 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   object_number            484956 non-null  object 
 1   is_highlight             484956 non-null  bool   
 2   is_timeline_work         484956 non-null  bool   
 3   is_public_domain         484956 non-null  bool   
 4   object_id                484956 non-null  int64  
 5   gallery_number           49541 non-null   object 
 6   department               484956 non-null  object 
 7   accessionyear            481094 non-null  object 
 8   object_name              482690 non-null  object 
 9   title                    456292 non-null  object 
 10  culture                  208190 non-null  object 
 11  period                   91143 non-null   object 
 12  dynasty                  23201 non-null   object 
 13  reign                    11236 non-null   object 
 14  port

In [12]:
df.isnull().sum()

object_number                   0
is_highlight                    0
is_timeline_work                0
is_public_domain                0
object_id                       0
gallery_number             435415
department                      0
accessionyear                3862
object_name                  2266
title                       28664
culture                    276766
period                     393813
dynasty                    461755
reign                      473720
portfolio                  458442
constituent_id             202443
artist_role                202443
artist_prefix              202443
artist_display_name        202443
artist_display_bio         204533
artist_suffix              202491
artist_alpha_sort          202443
artist_nationality         202443
artist_begin_date          202443
artist_end_date            202443
artist_gender              378474
artist_ulan_url            257515
artist_wikidata_url        260754
object_date                 13431
object_begin_d

### My suggestions for columns to keep and drop

**Keep**

is_highlight, is_timeline, object_id, department, accessionyear, object_name, title, culture, period, dynasty, reign, portfolio, artist_role, artist_pref, artist_displa, artist_nat, artist_begi, artist_end_, artist_gender, object_begi, object_end_, medium, geography_t, city, state, county, country, region, subregion, locale, locus, excavation, river, classification, repository

**Remove**
-  object_number
    - formatting is inconsistent, object_id has a lot more consistent formatting
- gallery_number            
    - not applicable for our purposes and values are mostly NaN
-  is_public_domain         
    - has to do with copyright, not really applicable in our case
-  artist_display_bio      
    - The informatino in this bio is listed in the adjacent columns seperately
-  constituent_id
    - seems to be an internal ID reference
-  artist_ulan_url          
    - links to external web pages
-  artist_wikidata_url      
    - links to external web pages
- artist_suffix
    - If we are interested or need names, we can just use the display name
- artist_alpha_sort
    - If we are interested or need names, we can just use the display name
-  dimensions               
    - not relevant to our analysis and looks like it would be difficult to parse
-  credit_line              
    - not really relevant to our analysis, just states who they gave the painting to
-  rights_and_reproduction  
    - legal data, not relevant to our analysis
-  link_resource            
    - links to external web page
-  object_wikidata_url     
    - links to external web page 
-  tags                     
    - not relevant for our analysis
-  tags_aat_url             
    - links to external web pages
-  tags_wikidata_url        
    - links to external web pages
- object_date 
    - It displays both the object begin and end date (for example begin: 1909, end: 1927) as 1909-27. It would be easier to just work with the begin and end date columns seperately.
- metadata_date
    - all values are NaN values

## Cleaning

In [13]:
# Drop columns

df = df.drop(columns = ['object_number','gallery_number', 'is_public_domain', 'artist_display_bio',
                       'constituent_id', 'artist_ulan_url', 'artist_wikidata_url',
                       'artist_suffix', 'artist_alpha_sort', 'dimensions', 'credit_line',
                       'rights_and_reproduction', 'link_resource', 'object_wikidata_url',
                       'tags', 'tags_aat_url', 'tags_wikidata_url', 'object_date', 
                        'metadata_date'])

In [14]:
df.shape

(484956, 35)

In [15]:
df.head(1)

,is_highlight,is_timeline_work,object_id,department,accessionyear,object_name,title,culture,period,dynasty,reign,portfolio,artist_role,artist_prefix,artist_display_name,artist_nationality,artist_begin_date,artist_end_date,artist_gender,object_begin_date,object_end_date,medium,geography_type,city,state,county,country,region,subregion,locale,locus,excavation,river,classification,repository
0,False,False,1,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,NaN,NaN,NaN,NaN,NaN,Maker,,James Barton Longacre,American,1794,1869,NaN,1853,1853,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"


In [16]:
# List of columns to split on '|'
columns_to_split = [
    'artist_display_name',
    'artist_role',
    'artist_prefix',
    'artist_nationality',
    'artist_begin_date',
    'artist_end_date',
    'artist_gender'
]

# Perform the split and assign to new columns
for col in columns_to_split:
    df[[f'{col}_1', f'{col}_2']] = df[col].str.split('|', n=1, expand=True)

# Drop the original unsplit columns
df = df.drop(columns=columns_to_split)

In [17]:
# Reorder columns

df = df[['object_id', 'is_highlight', 'is_timeline_work',
        'department', 'accessionyear', 'object_name', 'title', 'object_begin_date', 
         'object_end_date', 'artist_display_name_1', 'artist_role_1', 
         'artist_prefix_1', 'artist_nationality_1', 'artist_begin_date_1',
         'artist_end_date_1', 'artist_gender_1', 'artist_display_name_2', 'artist_role_2',
         'artist_prefix_2', 'artist_nationality_2', 'artist_begin_date_2',
         'artist_end_date_2', 'artist_gender_2', 'culture', 'period', 'dynasty', 
         'reign', 'portfolio', 'medium', 'geography_type', 'city', 'state', 'county',
         'country', 'region', 'subregion', 'locus', 'excavation', 'river',
         'classification', 'repository']]

In [18]:
df.head() #

,object_id,is_highlight,is_timeline_work,department,accessionyear,object_name,title,object_begin_date,object_end_date,artist_display_name_1,artist_role_1,artist_prefix_1,artist_nationality_1,artist_begin_date_1,artist_end_date_1,artist_gender_1,artist_display_name_2,artist_role_2,artist_prefix_2,artist_nationality_2,artist_begin_date_2,artist_end_date_2,artist_gender_2,culture,period,dynasty,reign,portfolio,medium,geography_type,city,state,county,country,region,subregion,locus,excavation,river,classification,repository
0,1,False,False,The American Wing,1979.0,Coin,One-dollar Liberty Head Coin,1853,1853,James Barton Longacre,Maker,,American,1794,1869,NaN,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
1,2,False,False,The American Wing,1980.0,Coin,Ten-dollar Liberty Head Coin,1901,1901,Christian Gobrecht,Maker,,American,1785,1844,NaN,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
2,3,False,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
3,4,False,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
4,5,False,False,The American Wing,1967.0,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"


In [19]:
# Check format of date columns
date_cols = [
    'accessionyear',
    'artist_begin_date_1',
    'artist_end_date_1',
    'artist_begin_date_2',
    'artist_end_date_2',
    'object_begin_date',
    'object_end_date'
]

df[date_cols].dtypes

accessionyear          object
artist_begin_date_1    object
artist_end_date_1      object
artist_begin_date_2    object
artist_end_date_2      object
object_begin_date       int64
object_end_date         int64
dtype: object

In [20]:
# Convert date columns to Int64 (to preserve NaN values) 
def extract_year(val):
    if pd.isna(val):
        return np.nan
    try:
        if isinstance(val, (pd.Timestamp, np.datetime64)):
            return val.year
        elif isinstance(val, str) and val.strip().isdigit():
            return int(val)
        elif isinstance(val, (float, int)):
            return int(val)
        elif isinstance(val, str):
            parsed = pd.to_datetime(val, errors='coerce')
            if pd.notna(parsed):
                return parsed.year
    except:
        return np.nan
    return np.nan

# Apply it to your year columns
for col in date_cols:
    df[col] = df[col].apply(extract_year).astype('Int64')


In [21]:
df.head()

,object_id,is_highlight,is_timeline_work,department,accessionyear,object_name,title,object_begin_date,object_end_date,artist_display_name_1,artist_role_1,artist_prefix_1,artist_nationality_1,artist_begin_date_1,artist_end_date_1,artist_gender_1,artist_display_name_2,artist_role_2,artist_prefix_2,artist_nationality_2,artist_begin_date_2,artist_end_date_2,artist_gender_2,culture,period,dynasty,reign,portfolio,medium,geography_type,city,state,county,country,region,subregion,locus,excavation,river,classification,repository
0,1,False,False,The American Wing,1979,Coin,One-dollar Liberty Head Coin,1853,1853,James Barton Longacre,Maker,,American,1794,1869,NaN,None,None,None,None,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
1,2,False,False,The American Wing,1980,Coin,Ten-dollar Liberty Head Coin,1901,1901,Christian Gobrecht,Maker,,American,1785,1844,NaN,None,None,None,None,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
2,3,False,False,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
3,4,False,False,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
4,5,False,False,The American Wing,1967,Coin,Two-and-a-Half Dollar Coin,1909,1927,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Gold,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"


In [22]:
df.dtypes

object_id                 int64
is_highlight               bool
is_timeline_work           bool
department               object
accessionyear             Int64
object_name              object
title                    object
object_begin_date         Int64
object_end_date           Int64
artist_display_name_1    object
artist_role_1            object
artist_prefix_1          object
artist_nationality_1     object
artist_begin_date_1       Int64
artist_end_date_1         Int64
artist_gender_1          object
artist_display_name_2    object
artist_role_2            object
artist_prefix_2          object
artist_nationality_2     object
artist_begin_date_2       Int64
artist_end_date_2         Int64
artist_gender_2          object
culture                  object
period                   object
dynasty                  object
reign                    object
portfolio                object
medium                   object
geography_type           object
city                     object
state   

In [23]:
object_types = df['object_name'].unique()
print(*object_types)

Coin Peso Centavos Pesos Bust Clock Vase Side Chair Figure Advertisement Ale glass Andiron Apothecary jar Statuette Frame Architectural elements Loggia elements Bedroom Argand lamp Lamp shade Armchair Banister-back armchair Drawing Slat-back armchair Spindle-back armchair Teapot Rocking chair Reclining armchair Folding armchair Side chair Revolving armchair Furniture hardware Astral lamp Window Aviary Baby's bottle Badge Baking dish Balcony Ball Wall panel Baluster Banjo clock Bank Baptismal basin Baptismal bowl Baptismal certificate Bottle Mug Keg Basin Wash basin Basin Stand Basin stand Basket Door Oil cruet Beaker Pulque beaker Tumbler Bean pot Folding bedstead Bedstead Bell Bell pull Bellows Bench Sleigh seat Painting Cameo medallion, minature Mask cast Medallion Painting, miniature Portrait Panel Plaque Bust, miniature Berry dish Spoon Betty lamp Bible Bill holder Fountain Birdhouse Birth and baptismal certificate, Fraktur Certificate, Fraktur Birth certificate, Fraktur Birth, bap

 Blade and Mounting for a dagger (Tantō) Blade  for a dagger (Tantō) Hatchet (Nata) Axe (tabar) with stiletto Saddle axe Sword (Shamshir) Sword (shamshir) with scabbard Sword (Shashqa) with scabbard Sword (Kilij) with scabbard Sword (Dha) with scabbard Short sword (Yatagan) with scabbard Dagger (Dha) with sheath Sword with scabbard and compass Sword with scabbard and baldric Two-handed sword with scabbard Sword (Dao) with scabbard and baldric Hilt of a gauntlet sword Sheath for a kris Sheath and knife Scabbard Kris sheath Scabbard for a dagger Knife sheath Two scabbard mounts Chape for a dagger sheath Sheath for a dagger Blade and mounting for a double-edged sword (Ken) Saddle (<i>Kura</i>) Fireman's helmet (Kajikabuto) Blade and for a hatchet (Nata) and mounting for a dagger (Chiisagatana-goshirae) Sword cane with two blades and sheath Hunting sword (Shamshir shikargar) with modern scabbard Sword with scabbard and belt Sword (Klewang) with scabbard Sword (shamshir) Sword (talwar) Swor

 Base design Belt buckle and plaques Vessel (Kendi) Scuplture Palanquin cover Inscriptions Fan handle and finial Sake flask Netsuke and ojime Skirt cloth Palampore Hip wrapper Partial hip wrapper (Tuppotiya) Sari Sari with blouse piece Illustrated single work One of a pair of armchairs Ceremonial hanging Waist cloth Jacket (Chabaori) Taxidermied deer Altar set Sari panel Box lid fragment (?) Jar fragment Patolu Two leaves from an album Album of twelve leaves Set of three hanging scrolls panel Hanging scroll remounted on a panel Sutra illustrated books illustrated book Casket panels Stomacher Shoe Buckles Cuffs Sash Veil Pelerine Fichu Evening dress Ensemble Suit Dress Maillot Evening ensemble Bikini Beachwear Sweater Evening suit Evening shoes Pumps Athletic shoes Shirt Shorts Athletic shorts Jockstrap Athletic jumpsuit Bathing trunks Athletic tights Sweat Pants Athletic shirt Shoes Fancy dress costume Boots Romper Evening pumps Jewelry Set Necktie Raincoat Underwear Jeans Socks Dinner

 Trick bottle or cup Stirring rod Rod Lentoid perfume bottle Bottle with three feet Bottle in the form of a bird Bottle, four-sided Bottle, fusiform Flask, indented Beaker, indented Bowl, Rippenschale Bowl, ribbed Eggshell Beaker or lamp Alabastron Millefiori plaque Base fragment Beads, 134 Flask with stylized grape cluster Flask, double head-shaped Amphoriskos Unguentarium Spindle whorl Krateriskos Amphora, pointed Vase, spindle-shaped Pyxis, inscribed Oinochoe, fragmentary Bell with handle in form of human figure Cup in the form of a monkey face Torch-holder Juglet Jug, miniature Krater Lagynos Scyphus Skyphos Strainer jug Askos Vase, miniature Hydria Flask, lentoid Pyxis lid Dinos Kylix Oinochoe Chariot wheel Jug, barrel-shaped Jug, barrel-shaped, with strainer Flask, head-shaped Wall bracket with bull's head Jug with female head Trick vase Baby feeder Composite lid Kylix with lid Jug, trefoil Ring-vase Flask, ring-shaped Ring-kernos Kernos Jug, spouted Bowl, miniature Krater, minia

 Beads, 13 Beads, 9 Millefiori beads, 20 Millefiori bead Counter or inlay Beads, 6 Bead, gold glass Beads, 11 Beads, 29 Millefiori pinhead Beads, 3 Beads, 17 Beads, 12 Bowl, network Pendant in the form of a bunch of grapes Pendant with female masks Counter or gaming piece Necklace with 15 beads Ornament cone Beads, 30 Beads, 16 Beads, 10 Beads, 22 Beads, 23 Beads - 33 Beads, 8 Beads, 42 Beads, 38 Beads, 51 Beads, 97 Beads, 20 Beads, 19 Beads, 33 Beads, 24 Face bead Beads, 54 Head pendant of a demon Hydriske Flask, neck fragment Head of Asklepios? Bowl, molded Bottle in the form of a man Jug in the shape of a head of an African Cup with base Jug with movable ring on neck Relief with fish Cup, "modiolus" Chevron bead Necklace with 54 millefiori beads Beads, 31 Millefiori bead rods, 15 Vase, onyx Head of Zeus Sarapis Roundel, medallion with head of Herakles Dish or plaque fragments Statuette of a figure Statue of Astarte ? Statuette fragment of Venus Oinochoe relief, woman Handle of a lam

 Flatware set Painting; Collage Cheese tray Furniture Dish and cover Starbrick Light Print cabinet Installation Jardinière and Pedestal Ūd Pipe Organ Stringed Instrument Lyre Book Harmonium Clavichord Oboe Shichigenkin ( 七弦琴) Double Stopped Whistle Organ Pipe Primitive European Bow Drum Sticks Rectangular Octave Spinet Jew's Harp Violin Bow Mandora Mandoline Mandola Chitarra Battente Mandolin Pandurina Cavaco Keyboard Glockenspiel Bentside Spinet Cornemuse Guitar Cittern Lyre Guitar Pochette Rebab Guenbri Gumuri Gunibri Sanxian Erxian Qānūn Yangqin  (揚琴) Koto Square Piano Rabāb Fung Kam Po Lo Luo (gong) Xiao Luo (gliss gong) Bo Shin-Cha Cha Koh Yunluo ("Cloud Gong") Suona Haidi Lapa Zhajiao Bajiao Gu Ch'in Sian Bangu (drum) Tánggǔ Gu Hai-Lo ManTou Gu (drum) Chang-ku Huagu Nanbanzi (wood block) Sihu Erhu Ti Chin Hu Chin Yeh Hu Hu Hu Zheng Pipa Yueqin (moon lute) Huobusi Xiao (flute) Dizi Quanzi Pina Xiao Haotong (natural trumpet) Goqing (磬) Osobuki Dora Nikko Sho Furi Tsuzumi Choshi Wan

 Commemorative jug Drawing of William Thomas Parke (1762-1847) Chromatic Harp Whistle Jar Vina (ekanda vina) Flageolet in D (?) Keys Tamburi Angklung Soprano Dulcian Güira Transverse Flute in G Furin "Persian" Sitar Goje Rabāba Ketjapi Saravatī Vīnā Kamaica Saing Waing Ah-Pwe (figures of Burmese orchestra) Ceremonial Rattle Rattle (thafu ma luangu) Eleke Oko (side-blown horn) Akam (Pluriarc) Tambuyuk (Conch Shell Trumpet) Mute Viola d'Amore Chest Organ Snares Manguaré Mandolin Case Nyckelharpa Bow Gotchnag (koch'nak) Orpheoreon (reproduction) Group of 17 mouthpieces (2 w/ cases) Mandolino Pangwa English horn Cor Solo Pellet Bells (3) Beaters Transverse flute in D Flat Transverse flute in B Kerar (lyre) Archtop Mandolin Side-Blown Trumpets Tanjore Tambura (female) Tanjore Tāmbūra (male) Ekadandi Vina Sruti box Ankle Bells Lithographic Plates Side drum String Instrument Cornet Construction Galoubet Lithographic Plate Bass Recorder Gaohu Luiqin Lâvta Sāz-ī-Kāshmīr Bandura Table Cover Belt

 Linen mark, Masaharta Innermost lid of Menkheperre Mummy board, Tabakmut; D21-Nested-Coffins Model ball Measuring rod, fragment Seal, plaque, wine jar Pendants, petal Ornament, Rosette Pendant, palmette Pendant, grapes Amulet, Isis, Horus Amulet, King, child Amulet, Bes with tambourine Amulet, squatting child, king Ear stud (?) Mold, Akhenaten cartouche Dress ornament, daisy Pendant, date Pendant, lotus petals Mold, scarab back Mold, Bes amulet Amulet, Amun Beads (2) Jar, blue-painted, Malqata Blue-painted jar fragment, bull leaping Jar, round bottomed, red ware, two-handled Body covering, openwork, winged goddess fragment Tile, palace of Ramesses II, stairway Tile, palace of Ramesses II, platform Tile, palace of Ramesses II, headdress wig Seal, cowroid, Was inscription Seal, cowroid, nefer inscription Seal, cowroid, Maat Pendant, composite cornflower Pendant, cornflowers Amulet, Heb-Sed Necklace, scarabs, Menkheperre, Nebmaatre, grasshopper, frog Bead, daisy Beads, leaf Pendant, ring

,,,,asdfsdf Mekes Staff Ostracon, cursive hieroglyphs Cone, rectangular (2) impression, Djehutimose, wab priest Fragment, unknown, rod (?) Vessel fragment, bead Weight: 100 deben ? Vase core, obsidian Tile inlay fragment Cone, Davies 599, Aanu, Sennu, king's son, house mistress Weight: 120 debens Weight: 2 kedets ? Bracelet with 24 plaques Box knob, Tutankhamun String of 7 ring, 18 spheroid, 6 cylinder and 4 barrel beads Jar fragment, Apophis I, Herti, Ahmose I See 21.9.466 Neck see 21.9.509 See 21.9.553 See 21.9.608 , uninscribed scarab Mold, persea fruit Necklace, Wedjat eyes (6) once on string (removed) Statuette, standing goddess, Lower Egypt Ostracon, figured, cartouches, seated man Mummy Board, Gautsoshen Scarab, lion, uraeus, spiral Amulet, son of Horus String of disc and cylindrical beads Cylinder seal with cartouche of Amenemhat III Cylinder seal, Ahmose-Nefertari, God's Wife, king's sister, king's daughter Scarab, royal name, Amenemhat III Nimaatre, sema-tawy Cowroid, Thutmos

 Wax model Harp-lute Letter Ornament & Architecture Valentine ; print ; ephemera Comforter Printed matter Ornament & Architecture Printed Matter Ornament & Architecture Orrery Clock Combination flintlock pistol and sword Print, Cut Paper Cut paper; print Photograph; Cabinet Card Beggar's bowl Valentine ; print; ephemera Covered containers Powder Flasks Print, Book Cover Print, Leaflet Revolver with case and cleaning rod Rifle with box of sights Wheellock pistols Combination flintlock pistol and concealed dagger Repeating flintlock pistol Pair of flintlock pistols with square-drive key Pair of flintlock pistols with case and accessories Smallsword with scabbard and case Scrapbook of engraving designs for firearms ornament Print; Visiting card Print; Book page Printed matter; Print Three arrowheads Flintlock Wender Pistol Pectoral Cross Copy of a wall painting Book, pamphlet Stereograph; Diablerie Print Drawing Sculpture; Time-based Media Print, Trade Card Trade Card Sickle, restored 136

In [24]:
### Create new dataframe with painting objects
new_df_paint = df[df['object_name'].isin(['Painting'])]

In [25]:
### Count of only painting objects
len(new_df_paint)

6014

In [26]:
new_df_paint.head()

,object_id,is_highlight,is_timeline_work,department,accessionyear,object_name,title,object_begin_date,object_end_date,artist_display_name_1,artist_role_1,artist_prefix_1,artist_nationality_1,artist_begin_date_1,artist_end_date_1,artist_gender_1,artist_display_name_2,artist_role_2,artist_prefix_2,artist_nationality_2,artist_begin_date_2,artist_end_date_2,artist_gender_2,culture,period,dynasty,reign,portfolio,medium,geography_type,city,state,county,country,region,subregion,locus,excavation,river,classification,repository
407,466,False,False,The American Wing,1883,Painting,Plaque Portrait of Benjamin Franklin,1800,1883,Louis Benjamin Huber Defeuille,Maker,,,<NA>,9999,NaN,None,None,None,None,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Ivory,Made in,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
408,467,False,False,The American Wing,1883,Painting,Plaque Portrait of Benjamin Franklin,1776,1883,Jacques Thouron,Artist,After,,<NA>,9999,,Weyller,Artist,,,<NA>,9999,,NaN,NaN,NaN,NaN,NaN,Enamel,Made in,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
409,468,False,False,The American Wing,1883,Painting,Plaque Portrait of Benjamin Franklin,1776,1883,Jacques Thouron,Artist,After,,<NA>,9999,NaN,None,None,None,None,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Ivory,Made in,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
410,469,False,False,The American Wing,1883,Painting,Plaque Portrait of Benjamin Franklin,1776,1883,Jacques Thouron,Artist,After,,<NA>,9999,NaN,None,None,None,None,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Enamel,Made in,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"
411,470,False,False,The American Wing,1883,Painting,Plaque Portrait of Benjamin Franklin,1776,1883,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,<NA>,<NA>,NaN,NaN,NaN,NaN,NaN,NaN,Probably porcelain,Made in,NaN,NaN,NaN,France,NaN,NaN,NaN,NaN,NaN,NaN,"Metropolitan Museum of Art, New York, NY"


In [27]:
# Save dataframe as a new csv to use in future steps
new_df_paint.to_csv('CleanedMetObjects.csv', index = False)

## Create tables for nodes & edges

### Nodes
* Artists
* Departments
* Time Period
* Artworks

In [25]:
# Load in data

df = pd.read_csv('CleanedMetObjects.csv')

In [26]:
# List column names
df.columns

Index(['object_id', 'is_highlight', 'is_timeline_work', 'department',
       'accessionyear', 'object_name', 'title', 'object_begin_date',
       'object_end_date', 'artist_display_name_1', 'artist_role_1',
       'artist_prefix_1', 'artist_nationality_1', 'artist_begin_date_1',
       'artist_end_date_1', 'artist_gender_1', 'artist_display_name_2',
       'artist_role_2', 'artist_prefix_2', 'artist_nationality_2',
       'artist_begin_date_2', 'artist_end_date_2', 'artist_gender_2',
       'culture', 'period', 'dynasty', 'reign', 'portfolio', 'medium',
       'geography_type', 'city', 'state', 'county', 'country', 'region',
       'subregion', 'locus', 'excavation', 'river', 'classification',
       'repository', 'century', 'century_start_year', 'century_end_year',
       'department_id', 'time_period_id'],
      dtype='object')

In [27]:
# List unique values
def list_unique_values(df):
  """Prints unique values for each column in a Pandas DataFrame.

  Args:
    df: The Pandas DataFrame.
  """
  for col in df.columns:
    unique_values = df[col].unique()
    print(f"Column '{col}':")
    print(unique_values)
    print("-" * 20)
    
list_unique_values(df)

Column 'object_id':
[   466    467    468 ... 894493 895906 895946]
--------------------
Column 'is_highlight':
[False  True]
--------------------
Column 'is_timeline_work':
[False  True]
--------------------
Column 'department':
['The American Wing' 'Arms and Armor' 'Asian Art'
 'Modern and Contemporary Art' 'European Sculpture and Decorative Arts'
 'Photographs' 'Arts of Africa, Oceania, and the Americas'
 'Drawings and Prints' 'European Paintings' 'Islamic Art'
 'Robert Lehman Collection' 'The Cloisters' 'Medieval Art'
 'Musical Instruments']
--------------------
Column 'accessionyear':
[1883 1996 1966 1967 1913 1972 1939 1980 1912 1908 1891 1901 1956 1940
 1910 1987 1963 1929 1977 1881 1915 1965 1969 1906 1927 1982 1909 1979
 1907 1976 1933 1916 1974 1952 1890 1904 1957 1954 1971 1903 1921 1964
 1953 1914 1968 1893 1918 1932 1926 1949 1905 1922 1923 1995 2006 1962
 1973 1925 1955 1937 1941 1902 1892 1895 1978 1931 1943 1924 1959 2014
 1942 1917 1961 1877 1887 1948 1920 1919 1975 18

In [28]:
# Normalize Artists table
artist_1 = df[['artist_display_name_1']].rename(columns={'artist_display_name_1': 'artist_name'})
artist_2 = df[['artist_display_name_2']].rename(columns={'artist_display_name_2': 'artist_name'})

all_artists = pd.concat([artist_1, artist_2], ignore_index=True).dropna().drop_duplicates().reset_index(drop=True)
all_artists['artist_id'] = all_artists.index + 1

# Reorder columns: primary key first
all_artists = all_artists[['artist_id', 'artist_name']]
all_artists.to_csv("nodes_artists.csv", index=False)

In [29]:
# Normalize Departments table
departments = df[['department']].dropna().drop_duplicates().reset_index(drop=True)
departments['department_id'] = departments.index + 1
departments = departments[['department_id', 'department']]
departments.to_csv("nodes_departments.csv", index=False)

In [30]:
# Normalize Time Period table

# Create a century label with corresponding start/end years from object_begin_date
def get_century_info(year):
    if pd.isna(year):
        return ("Unknown", None, None)

    year = int(year)

    if year > 0:
        century = (year - 1) // 100 + 1
        start_year = (century - 1) * 100 + 1
        end_year = century * 100
        label = f"{century}th Century CE"
    elif year < 0:
        century = (-year) // 100 + 1
        start_year = -century * 100 + 1
        end_year = -(century - 1) * 100
        label = f"{century}th Century BCE"
    else:
        return ("1st Century CE", 1, 100)

    return (label, start_year, end_year)

# Apply and expand into three columns
df[['century', 'century_start_year', 'century_end_year']] = df['object_begin_date'].apply(
    lambda y: pd.Series(get_century_info(y))
)

# Build TimePeriod node table
time_periods = df[['century', 'century_start_year', 'century_end_year']] \
    .dropna(subset=['century']) \
    .drop_duplicates() \
    .reset_index(drop=True)

time_periods['time_period_id'] = time_periods.index + 1
time_periods = time_periods[['time_period_id', 'century', 'century_start_year', 'century_end_year']]
time_periods.to_csv("nodes_time_periods.csv", index=False)

In [31]:
# Normalize Artworks table
artworks = df[['object_id', 'title', 'object_begin_date', 'object_end_date', 'department', 'century', 'accessionyear']]
artworks = artworks.dropna(subset=['object_id']).drop_duplicates(subset=['object_id']).reset_index(drop=True)
artworks.to_csv("nodes_artworks.csv", index=False)

### Edges
* Artwork to Artist
* Artwork to Department
* Artwork to Time Period

In [32]:
# Artwork to Artist
# Use your artist name → ID mapping from earlier
artist_name_to_id = dict(zip(all_artists['artist_name'], all_artists['artist_id']))

artwork_artist_edges = []

for _, row in df.iterrows():
    for col in ['artist_display_name_1', 'artist_display_name_2']:
        artist_name = row[col]
        if pd.notna(artist_name):
            artwork_artist_edges.append({
                'object_id': row['object_id'],
                'artist_id': artist_name_to_id.get(artist_name)
            })

pd.DataFrame(artwork_artist_edges).dropna().to_csv("edges_artwork_artist.csv", index=False)

In [33]:
# Artwork to Department
department_name_to_id = dict(zip(departments['department'], departments['department_id']))

df['department_id'] = df['department'].map(department_name_to_id)
artwork_department_edges = df[['object_id', 'department_id']].dropna()
artwork_department_edges.to_csv("edges_artwork_department.csv", index=False)

In [34]:
# Artwork to Time Period
time_period_name_to_id = dict(zip(time_periods['century'], time_periods['time_period_id']))

df['time_period_id'] = df['century'].map(time_period_name_to_id)
artwork_time_edges = df[['object_id', 'time_period_id']].dropna()
artwork_time_edges.to_csv("edges_artwork_time_period.csv", index=False)

### Visit graph_setup for next steps (individual steps)